In [1]:
import sqlite3
import pandas as pd
import csv

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [2]:
# These helper functions will be useful as we work
# with the SQLite database from python

DB = "sal.db"

def run_query(q):
    with sqlite3.connect(DB) as conn:
        return pd.read_sql(q,conn)

def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

In [3]:
salary1 = pd.DataFrame({'employee_id' : [1,1,2,2],
                 'salary_amt' : [50000,70000,70000,90000],
                 'from_dt' : [pd.Timestamp('2019-1-1'), pd.Timestamp('2019-4-1'),
                              pd.Timestamp('2019-1-1'),pd.Timestamp('2019-3-1')],
                 'to_dt' : [pd.Timestamp('2019-2-1'), pd.Timestamp('2100-1-1'),
                            pd.Timestamp('2019-5-1'),pd.Timestamp('2100-1-1')]})
# Using date 01.01.5999 raises an error:
# OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 5999-01-01 00:00:00
salary1

,employee_id,salary_amt,from_dt,to_dt
0,1,50000,2019-01-01,2019-02-01
1,1,70000,2019-04-01,2100-01-01
2,2,70000,2019-01-01,2019-05-01
3,2,90000,2019-03-01,2100-01-01


In [4]:
salary2 = pd.DataFrame({'employee_id' : [1,1,2,2],
                 'salary_amt' : [50000,70000,70000,90000],
                 'from_dt' : [pd.Timestamp('2019-1-1'), pd.Timestamp('2019-4-1'),
                              pd.Timestamp('2019-1-1'),pd.Timestamp('2019-5-1')],
                 'to_dt' : [pd.Timestamp('2019-3-31'), pd.Timestamp('2100-1-1'),
                            pd.Timestamp('2019-4-30'),pd.Timestamp('2100-1-1')]})
salary2

,employee_id,salary_amt,from_dt,to_dt
0,1,50000,2019-01-01,2019-03-31
1,1,70000,2019-04-01,2100-01-01
2,2,70000,2019-01-01,2019-04-30
3,2,90000,2019-05-01,2100-01-01


In [5]:
position = pd.DataFrame({'employee_id' : [1,1,1,3],
                         'position_nm' : ['DQ', 'Lead DQ', 'PM', 'DQ'],
                         'from_dt' : [pd.Timestamp('2019-2-1'), pd.Timestamp('2019-4-1'),
                              pd.Timestamp('2019-9-1'),pd.Timestamp('2019-5-1')],
                         'to_dt' : [pd.Timestamp('2019-3-31'), pd.Timestamp('2019-8-31'),
                            pd.Timestamp('2100-1-1'),pd.Timestamp('2100-1-1')]})
position

,employee_id,position_nm,from_dt,to_dt
0,1,DQ,2019-02-01,2019-03-31
1,1,Lead DQ,2019-04-01,2019-08-31
2,1,PM,2019-09-01,2100-01-01
3,3,DQ,2019-05-01,2100-01-01


In [6]:
trxn = pd.DataFrame({'account_id' : [1,1,1,2,2],
                         'trxn_dt' : [pd.Timestamp('2019-11-1'), pd.Timestamp('2019-11-3'),
                                      pd.Timestamp('2019-11-5'),pd.Timestamp('2019-11-1'),
                                      pd.Timestamp('2019-11-11')],
                         'trxn_amt' : [1000,-300,500,500,-500]
                         })
trxn

,account_id,trxn_dt,trxn_amt
0,1,2019-11-01,1000
1,1,2019-11-03,-300
2,1,2019-11-05,500
3,2,2019-11-01,500
4,2,2019-11-11,-500


In [7]:
balance = pd.DataFrame({'account_id' : [1,1,1,2,2],
                         'balance_dt' : [pd.Timestamp('2019-11-1'), pd.Timestamp('2019-11-2'),
                                      pd.Timestamp('2019-11-3'),pd.Timestamp('2019-11-2'),
                                      pd.Timestamp('2019-11-3')],
                         'balance_amt' : [1000,1000,500,300,0]
                         })
balance

,account_id,balance_dt,balance_amt
0,1,2019-11-01,1000
1,1,2019-11-02,1000
2,1,2019-11-03,500
3,2,2019-11-02,300
4,2,2019-11-03,0


In [8]:
tables = {
    'salary1': salary1,
    'salary2': salary2,
    'position': position,
    'trxn' : trxn,
    'balance':balance
    }

with sqlite3.connect(DB) as conn:    
    for name, data in tables.items():
        conn.execute("DROP TABLE IF EXISTS {};".format(name))
        data.to_sql(name,conn,index=False)

In [9]:
salary_intersection_gap = '''
SELECT s1.employee_id, s1.from_dt, 'gap' AS type 
FROM salary1 AS s1, salary1 AS s2
WHERE s1.employee_id = s2.employee_id AND s1.from_dt > s2.to_dt
UNION
SELECT s1.employee_id, s1.from_dt, 'intersection' AS type 
FROM salary1 as s1, salary1 AS s2
WHERE s1.employee_id = s2.employee_id AND s1.from_dt > s2.from_dt AND s2.to_dt >= s1.from_dt 
AND s1.to_dt != '2100-01-01'
'''
run_query(salary_intersection_gap)

,employee_id,from_dt,type
0,1,2019-04-01 00:00:00,gap
1,2,2019-03-01 00:00:00,intersection


In [10]:
salary_position = '''
SELECT p.employee_id, 
CASE 
    WHEN s.from_dt >  p.from_dt then s.from_dt
    ELSE p.from_dt
END AS from_dt,
CASE
    when s.to_dt < p.to_dt then s.to_dt
    else p.to_dt
END AS to_dt,
p.position_nm,s.salary_amt
FROM position p
INNER JOIN salary2 s 
ON s.employee_id = p.employee_id  and s.to_dt >= p.from_dt  and s.from_dt <= p.to_dt
'''
run_query(salary_position)

,employee_id,from_dt,to_dt,position_nm,salary_amt
0,1,2019-02-01 00:00:00,2019-03-31 00:00:00,DQ,50000
1,1,2019-04-01 00:00:00,2019-08-31 00:00:00,Lead DQ,70000
2,1,2019-09-01 00:00:00,2100-01-01 00:00:00,PM,70000


In [11]:
balance_in_the_end_of_a_day = '''
SELECT account_id, trxn_dt, 
sum(trxn_amt) over (partition by account_id order by trxn_dt) as balance
FROM trxn
'''
run_query(balance_in_the_end_of_a_day)

,account_id,trxn_dt,balance
0,1,2019-11-01 00:00:00,1000
1,1,2019-11-03 00:00:00,700
2,1,2019-11-05 00:00:00,1200
3,2,2019-11-01 00:00:00,500
4,2,2019-11-11 00:00:00,0


In [13]:
transaction_not_null = '''
SELECT pa.account_id, pa.balance_dt as trxn_dt, b.balance_amt - pa.previous_amt as trxn_amt
FROM balance AS b
INNER JOIN
    (
    SELECT account_id, balance_dt, balance_amt, 
    LAG(balance_amt, 1) OVER (partition by account_id order by balance_dt) AS previous_amt
    FROM balance
    ) pa 
ON b.account_id = pa.account_id  and b.balance_dt = pa.balance_dt and b.balance_amt != pa.previous_amt

'''
run_query(transaction_not_null)

,account_id,trxn_dt,trxn_amt
0,1,2019-11-03 00:00:00,-500
1,2,2019-11-03 00:00:00,-300
